# Introduction to APIs

* **Allows a provider to control what information you have access to, and how you access the data.**
* **Check any Terms of Use associated with the API**
* **Also check Rate Limits -> How many times you can ping their server (usually per hour)**

### We'll be collecting data from http://opendata.dc.gov/datasets/dc3289eab3d2400ea49c154863312434_8 which is a DC Crime dataset of crime in the last 30 days

### First, import all the things you need. 

In [1]:
import json              # Most API's provide data as a json file, 
import csv               # So we can save and read csv
import requests          # To access the API
import pandas as pd      # So we can work with the data in a dataframe
import time              # So we can control time.

### Initiate output csv file
* Next, we call into being a new .csv file, and assign it to the variable `'output_filename'`
* Then open the file using the `'csv'` module we imported above, and give it write privileges
* Last, we write in some column headers. These are the fields we want to fetch from the json. 

In [25]:
output_filename = 'ipython_foo.csv'
fileWriter = csv.writer(open(output_filename, "w+"),delimiter=",")
fileWriter.writerow(['anc', 'census_tract', 'coords', 'district', 'method', 'objectid', 'offense', 'report_time', 'shift', 'ward'])

81

### Grab raw JSON data from the API
* Assign the API URL to a variable name
* Using the requests module we imported, get the 30-day data 
* Let the code sleep for 5 seconds to allow time for the response to come back
* Check the response code:
    * for reference, response codes:  http://www.restapitutorial.com/httpstatuscodes.html
* Read in the response and assign it to a variable `'data'`

In [3]:
crime_incidents_30day_url = 'http://opendata.dc.gov/datasets/dc3289eab3d2400ea49c154863312434_8.geojson'

response = requests.get(crime_incidents_30day_url)
time.sleep(5)

print("\nThis is the response status: ", response.raise_for_status())

data = json.loads(response.text) #r.json()


This is the response status:  None


In [4]:
data.keys()

dict_keys(['type', 'features'])

* The JSON is a nested dictionary: the nest we care about is the `'features'` nest, so lets grab that part and assign it to data (essentially overwriting the first `'data'`)

In [5]:
data = data['features']

### Explore a little bit
* Now we can look to see what keys we have this `'features'` subset:
    - We have `'type'`, `'geometry'` and `'properties'`

In [6]:
data[0]   # what kind of data do we have in the first record?

{'geometry': {'coordinates': [-76.9975579843, 38.909409767], 'type': 'Point'},
 'properties': {'ANC': '5D',
  'BLOCKSITEADDRESS': '1300 - 1399 BLOCK OF 5TH STREET NE',
  'BLOCKXCOORD': 400212,
  'BLOCKYCOORD': 137949,
  'BLOCK_GROUP': '008803 1',
  'BUSINESSIMPROVEMENTDISTRICT': None,
  'CCN': '16089057',
  'CENSUS_TRACT': '008803',
  'DISTRICT': 'FIFTH',
  'END_DATE': '2016-06-01T20:50:01.000Z',
  'LASTMODIFIEDDATE': '2016-06-03T08:32:14.000Z',
  'METHOD': 'OTHERS',
  'NEIGHBORHOODCLUSTER': '23',
  'OBJECTID': 193290,
  'OFFENSE': 'THEFT F/AUTO',
  'PSA': '506',
  'REPORTDATETIME': '2016-06-02T17:03:13.000Z',
  'SHIFT': 'EVENING',
  'START_DATE': '2016-06-01T13:50:25.000Z',
  'VOTING_PRECINCT': 'Precinct 76',
  'WARD': '5'},
 'type': 'Feature'}

** How does this break down?**

In [7]:
data[0].keys()  # Top level keys are 'geometry' and 'properties'

dict_keys(['geometry', 'type', 'properties'])

In [8]:
data[0]['geometry'].keys()    # keys inside 'geometry' are:

dict_keys(['type', 'coordinates'])

In [9]:
data[0]['properties'].keys()  # keys inside 'properties' are:

dict_keys(['BLOCK_GROUP', 'START_DATE', 'DISTRICT', 'CENSUS_TRACT', 'BLOCKYCOORD', 'BLOCKXCOORD', 'OFFENSE', 'ANC', 'SHIFT', 'END_DATE', 'METHOD', 'VOTING_PRECINCT', 'BLOCKSITEADDRESS', 'BUSINESSIMPROVEMENTDISTRICT', 'CCN', 'NEIGHBORHOODCLUSTER', 'WARD', 'LASTMODIFIEDDATE', 'PSA', 'OBJECTID', 'REPORTDATETIME'])

### Organising the JSON into columns we want
* The columns need to be extracted in the same order as we have in our csv that we created in the beginning:
    - On the left we have the csv columns names - in lower case so we dont get confused
    - On the right we have the JSON column names that we handpicked after inspected the data above

* We will read in this function first, so we can call it later

In [10]:
def get_json_fields(data, i):

    row_data = {}

    row_data['anc'] = data[i]['properties']['ANC']
    row_data['census_tract'] = data[i]['properties']['CENSUS_TRACT']
    row_data['coords'] = data[i]['geometry']['coordinates']
    row_data['district'] = data[i]['properties']['DISTRICT']
    row_data['method'] = data[i]['properties']['METHOD']
    row_data['objectid'] = data[i]['properties']['OBJECTID']
    row_data['offense'] = data[i]['properties']['OFFENSE']
    row_data['report_time'] = data[i]['properties']['REPORTDATETIME']
    row_data['shift'] = data[i]['properties']['SHIFT']
    row_data['ward'] = data[i]['properties']['WARD']

    return row_data

### Extracted these colums
* Now we can call the function:
    - First we make an empty list to store each row or record of data
    - Then we say "for each record in the JSON (i), run the `'get_data'` function, and put the neat result in the `'data_list'` list

In [11]:
data_list = []
        
for i in range(len(data)):
    data_list.append(get_json_fields(data, i))

* Best check we have data in our list:
    - How many records is it?
    - What does the first record look like?

In [20]:
len(data_list)

3035

In [13]:
data_list[0]

{'anc': '5D',
 'census_tract': '008803',
 'coords': [-76.9975579843, 38.909409767],
 'district': 'FIFTH',
 'method': 'OTHERS',
 'objectid': 193290,
 'offense': 'THEFT F/AUTO',
 'report_time': '2016-06-02T17:03:13.000Z',
 'shift': 'EVENING',
 'ward': '5'}

* Great! Now we can make the list into a dataframe:

In [14]:
dataDF = pd.DataFrame(data_list)

In [15]:
dataDF.head()   # 'head()' just cos we only want to see the first few records to check it's ok

,anc,census_tract,coords,district,method,objectid,offense,report_time,shift,ward
0,5D,008803,"[-76.9975579843, 38.909409767]",FIFTH,OTHERS,193290,THEFT F/AUTO,2016-06-02T17:03:13.000Z,EVENING,5
1,3E,001001,"[-77.0853018915, 38.9599152513]",SECOND,OTHERS,193291,THEFT/OTHER,2016-06-02T17:39:32.000Z,EVENING,3
2,5C,009102,"[-76.9819666406, 38.9219389949]",FIFTH,OTHERS,193292,ROBBERY,2016-06-02T17:50:50.000Z,EVENING,5
3,5D,008904,"[-76.9741901124, 38.9014976093]",FIFTH,OTHERS,193293,THEFT/OTHER,2016-06-02T18:02:32.000Z,EVENING,5
4,6B,007000,"[-76.9955880526, 38.8827269275]",FIRST,OTHERS,193294,THEFT F/AUTO,2016-06-02T18:04:42.000Z,EVENING,6


### Append this data to our existing CSV file
* We created the CSV with our desired column headers already -> now we want to add the data
* The 'a' opens the file in 'append' mode
* Probably dont need the 'a' in both the `'open'` and the `'dataDF.to_csv'` lines, but hey!
* Header is False, cos we already have one
* Index is false, cos we dont want to add the Index column which is automatically created in Pandas dataframes to our CSV - all our columns will get thrown off!

In [26]:
with open(output_filename, 'a') as f:
    dataDF.to_csv(f, mode='a', header=False, index=False)

### If appending data, we should check we have no duplicates
* Since this is only 30 days, we might want to run the collection regularly to get updated information.
* Rather than creating a new file every time, we can just append the data to our existing file
* We could collect every 30 days and maybe not have any duplicates, but we might miss some records if our collection time is a little off
* We could collect every 29 days and not miss any data, but we might end up with duplicates. 
* Or, you could collect every _day_ to get daily updates
    - Here is a function `'drop_duplicates'` that keeps only the first record if a record is duplicated so that order is preserved (this may or may not be important, depending on what you're doing)
    
#### weird thing:
* Running drop_duplicates here in ipython notebook with `'temp.to_csv(output_filename, header=False, index=False)'` leads to a missing header (no column names). When run in spyder, I dont have this problem. Good thing we dont need to remove the header when we append to the existing csv so as not to duplicate the header row, since we are already removing duplicates :D

In [27]:
def drop_duplicates(output_filename):
        temp = pd.read_csv(output_filename)
        temp.drop_duplicates(keep='first', inplace=True)
        temp.to_csv(output_filename, index=False)

In [28]:
drop_duplicates(output_filename)

### IMPROVEMENTS
#### Dealing with duplicates

* A record can be updated when new information comes in, leading to a new record line with only the `'objectid'` being duplicated
* If this is the case, you'll want to keep the `'_last_'` duplicate, as this will the updated one
* Also if this is the case, you'll want to check duplicates based _only_ on the `'objectid'` column, as this is the only column that will not be updated.
* There is a way to `'drop_duplicates'` in pandas based on whether there is duplication in a specifed column (eg `'objectid'` but this resulted in a keyerror for me...I'll need to figure it out :(

#### Collected pre-filtered 30-day data
* The API allows you to grab filtered JSON data. Potentially, you could use this to only gather data beginning where your previous collection left off - no duplicates! OR collect only incidents involving a gun. However, when I attempted this, the API `'response'` was a 202, meaning it was accepted, but the data itself was pending...so if my requests connection stays open (only if I'm running off a server) I might receive the data within 24 hours? Who knows. 

### Scheduling to run automatically every given number of hours/minutes/seconds...

Take a look at  [scheduled_API_code.py](http/...)